# Análise e Exploração de Dados - Camada Gold

**Autor:** Marcelo Veras <br>
**Data:** 30 de Julho de 2025

## 1. Introdução

Este notebook fornece uma interface interativa para consultar e visualizar os dados no Data Warehouse, que foi populado pelo pipeline de ETL.

O objetivo é permitir a validação dos dados e a exploração direta do modelo relacional, sem a necessidade de um cliente de banco de dados externo. O código abaixo se conecta diretamente ao banco de dados SQL Server que está rodando no ambiente Docker e utiliza a biblioteca `pandas` para executar consultas SQL e exibir os resultados.

In [17]:
import pandas as pd
from sqlalchemy import create_engine
import urllib

server = 'desafio_db'
database = 'DesafioDB'
username = 'sa'
password = 'CocoBambuCBLAB123@@'

params = urllib.parse.quote_plus(
    f"DRIVER={{ODBC Driver 18 for SQL Server}};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    f"TrustServerCertificate=yes;"
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

try:
    connection = engine.connect()
    print(" Conexão com o Data Warehouse (SQLAlchemy) bem-sucedida!")
    connection.close()
except Exception as e:
    print(f" Falha na conexão: {e}")

 Conexão com o Data Warehouse (SQLAlchemy) bem-sucedida!


## 2. Consultas de Exemplo

Abaixo estão algumas consultas SQL de exemplo para explorar os dados carregados. Você pode executar cada célula de código para ver o resultado.

### Consulta 1: Visualizar os 5 primeiros pedidos na tabela principal

In [19]:
print("--- Tabela: Pedidos (5 primeiros registros) ---")

sql_query_pedidos = "SELECT TOP 5 * FROM Pedidos;"
df_pedidos = pd.read_sql(sql_query_pedidos, engine)
display(df_pedidos)

--- Tabela: Pedidos (5 primeiros registros) ---


,guest_check_id,restaurante_id_fk,funcionario_id_fk,numero_pedido,data_negocio_aberto,data_abertura_utc,data_fechamento_utc,fechado,total_pedido,total_desconto,total_pago,numero_mesa
0,112233235,2,2,9999,2025-01-01,2025-01-01 09:09:09+00:00,2025-01-01 12:12:12+00:00,True,150.8,-10.0,150.8,80
1,223344556,5,4,8888,2025-02-15,2025-02-15 10:00:00+00:00,2025-02-15 11:30:00+00:00,True,180.5,-5.0,180.5,22
2,334455667,4,3,7777,2025-03-10,2025-03-10 18:45:00+00:00,2025-03-10 20:00:00+00:00,True,250.0,-20.0,250.0,17B
3,445566778,3,1,6666,2025-04-01,2025-04-01 07:00:00+00:00,2025-04-01 08:30:00+00:00,True,75.0,0.0,75.0,12A
4,1122334455,1,1,1234,2024-01-01,2024-01-01 09:09:09+00:00,2024-01-01 12:12:12+00:00,True,109.9,-10.0,109.9,90


### Consulta 2: Listar todos os restaurantes e funcionários cadastrados

In [7]:
print("\n--- Tabela: Restaurantes ---")
display(pd.read_sql("SELECT * FROM Restaurantes;", engine))

print("\n--- Tabela: Funcionarios ---")
display(pd.read_sql("SELECT * FROM Funcionarios;", engine))


--- Tabela: Restaurantes ---


,restaurante_id,loc_ref
0,1,99 CB CB
1,2,777 CB CB
2,3,555 EE XX
3,4,999 DD YY
4,5,888 AA ZZ



--- Tabela: Funcionarios ---


,funcionario_id,numero_funcionario,nome_completo,cargo
0,1,55555,Funcionario 55555,Garçom
1,2,77777,Funcionario 77777,Garçom
2,3,99999,Funcionario 99999,Garçom
3,4,88888,Funcionario 88888,Garçom


### Consulta 3: Contar o número de pedidos por restaurante

In [9]:
print("\n--- Análise: Contagem de pedidos por restaurante ---")

sql_query_join = """
SELECT 
    r.loc_ref AS nome_restaurante,
    COUNT(p.guest_check_id) AS total_pedidos
FROM 
    Pedidos AS p
JOIN 
    Restaurantes AS r ON p.restaurante_id_fk = r.restaurante_id
GROUP BY 
    r.loc_ref
ORDER BY 
    total_pedidos DESC;
"""

df_pedidos_por_restaurante = pd.read_sql(sql_query_join, engine)
display(df_pedidos_por_restaurante)


--- Análise: Contagem de pedidos por restaurante ---


,nome_restaurante,total_pedidos
0,555 EE XX,1
1,777 CB CB,1
2,888 AA ZZ,1
3,99 CB CB,1
4,999 DD YY,1


### Consulta 4: Pedidos por Funcionário e Restaurante

In [22]:
print("\n--- Análise: Detalhamento de Pedidos por Funcionário e Restaurante ---")

sql_query_pedidos_completos = """
SELECT
    p.guest_check_id,
    p.numero_pedido,
    r.loc_ref AS nome_restaurante,
    f.nome_completo AS nome_funcionario,
    p.total_pedido,
    p.fechado,
    p.data_abertura_utc
FROM 
    Pedidos AS p
JOIN 
    Funcionarios AS f ON p.funcionario_id_fk = f.funcionario_id
JOIN
    Restaurantes AS r ON p.restaurante_id_fk = r.restaurante_id
ORDER BY
    p.data_abertura_utc DESC;
"""

df_pedidos_completos = pd.read_sql(sql_query_pedidos_completos, engine)

display(df_pedidos_completos)


--- Análise: Detalhamento de Pedidos por Funcionário e Restaurante ---


,guest_check_id,numero_pedido,nome_restaurante,nome_funcionario,total_pedido,fechado,data_abertura_utc
0,445566778,6666,555 EE XX,Funcionario 55555,75.0,True,2025-04-01 07:00:00+00:00
1,334455667,7777,999 DD YY,Funcionario 99999,250.0,True,2025-03-10 18:45:00+00:00
2,223344556,8888,888 AA ZZ,Funcionario 88888,180.5,True,2025-02-15 10:00:00+00:00
3,112233235,9999,777 CB CB,Funcionario 77777,150.8,True,2025-01-01 09:09:09+00:00
4,1122334455,1234,99 CB CB,Funcionario 55555,109.9,True,2024-01-01 09:09:09+00:00


### 3. Faça Suas Próprias Consultas!

Sinta-se à vontade para usar a célula de código abaixo para escrever e executar suas próprias consultas SQL no Data Warehouse.

In [20]:
# Escreva sua consulta SQL aqui dentro das três aspas
sua_query = """
SELECT * FROM Impostos_Pedidos;
"""

# Executa a consulta e exibe o resultado
try:
    df_resultado = pd.read_sql(sua_query, engine)
    display(df_resultado)
except Exception as e:
    print(f"Erro ao executar a consulta: {e}")


,imposto_pedido_id,guest_check_id_fk,numero_imposto,total_venda_tributavel,total_imposto_cobrado,taxa_imposto
0,1002,1122334455,28,119.9,20.81,21.0
1,1003,112233235,21,200.8,25.99,11.0
2,1004,445566778,18,75.0,9.00,12.0
3,1005,334455667,30,250.0,30.00,12.0
4,1006,223344556,11,180.5,21.66,12.0
